In [16]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import imageio
from PIL import Image


In [17]:
def create_directory(directory_name):
    '''Create a new directory and verify if it doesn't already exist
    '''

    try:
        # Create a directory
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")

In [18]:
IMAGE_DIR = "../dataset/training/images/"
IMAGE_DIR_AUGMENTED = "../dataset/training/augmented_images/"
GT_DIR = "../dataset/training/groundtruth/"
GT_DIR_AUGMENTED = "../dataset/training/augmented_groundtruth/"


# Creates the augmented directory if it doesn't already exist
create_directory(IMAGE_DIR_AUGMENTED)
create_directory(GT_DIR_AUGMENTED)

Directory '../dataset/training/augmented_images/' created successfully.
Directory '../dataset/training/augmented_groundtruth/' created successfully.


### Helper function

In [19]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def load_image_web(url):
    ### THIS DOES NOT WORK !!!
    ###data = np.array(PIL.Image.open(urllib.request.urlopen(url)))
    # We use that even if it is deprecated
    data = mpimg.imread(url)
    return data


def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg


# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:, :, 0] = gt_img8
        gt_img_3c[:, :, 1] = gt_img8
        gt_img_3c[:, :, 2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg


def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if is_2d:
                im_patch = im[j : j + w, i : i + h]
            else:
                im_patch = im[j : j + w, i : i + h, :]
            list_patches.append(im_patch)
    return list_patches

def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:, :, 0] = predicted_img * 255

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, "RGB").convert("RGBA")
    overlay = Image.fromarray(color_mask, "RGB").convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

def save_data_augmented(augmented_image, augmented_groundtruth):
    '''Save the new data generated by augmentation
    Arguments :
        - augmented_image :
        - augmented_groundtruth : 
    '''


    files_augmented = os.listdir(IMAGE_DIR_AUGMENTED)
    a = len(files_augmented)

    files = os.listdir(IMAGE_DIR)
    n = len(files)

    # Path with the name
    image_name = IMAGE_DIR_AUGMENTED + "satImage_" + str(a+1).zfill(3) + ".png"
    groundtruth_name = GT_DIR_AUGMENTED + "satImage_" + str(a+1).zfill(3) + ".png"

    # Save the image and the groundtruth in the augmented directories
    Image.fromarray((augmented_image * 255).astype(np.uint8)).save(image_name)
    Image.fromarray(augmented_groundtruth).save(groundtruth_name)

### Loading the images

In [20]:
files = os.listdir(IMAGE_DIR)
n = min(100, len(files))  # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(IMAGE_DIR + files[i]) for i in range(n)]
print(files[0])

print("Loading " + str(n) + " images")
gt_imgs = [load_image(GT_DIR + files[i]) for i in range(n)]
print(files[0])


Loading 100 images
satImage_001.png
Loading 100 images
satImage_001.png


### Data augmentation

In [21]:
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage


In [22]:
def data_augmentation_flip(image, groundtruth, horonzital = True):
    '''Create new image by flipping the image
    Arguments : 
        - image : input image of shape (H,W) that we want to augment
        - groundtruth : segmentation map of the corresponding input image
        - horonzital : flip horizontally the image if True, flip vertically the image if False
    Return :
        - flipped_image : images flipped 
        - flipped_groundtruth : associated segmentation map flipped
    '''

    # Shape of the image on which the segmentation map is placed
    # The shape of the segmentation map array is identical to the image shape
    shape_image = groundtruth.shape  # (400,400)

    # Object representing a segmentation map associated with an image
    seg_map =  SegmentationMapsOnImage(groundtruth, shape=shape_image)

    # Flip the image
    if horonzital :
        seq = iaa.Sequential([
                            iaa.flip.HorizontalFlip(1)  # Flip the image horizontally
                            ]) 
    else : 
        seq = iaa.Sequential([
                            iaa.flip.VerticalFlip(1)  # Flip the image vertically
                            ])
    
    # Flipping the image
    flipped_image, flipped_groundtruth = seq(image=image, segmentation_maps=seg_map)
    # The flipped segmentation map is converted back to an array
    flipped_groundtruth = 255 * flipped_groundtruth.get_arr().astype('uint8')

    return flipped_image, flipped_groundtruth

In [23]:
def data_augmentation_rotation(image, groundtruth, degree = 90):
    '''Create new image by image rotation
    Arguments : 
        - image : input image of shape (H,W) that we want to augment
        - groundtruth : segmentation map of the corresponding input image
        - degree : value of the degree of rotation
    Return :
        - augmented_image : images rotated 
        - augmented_groundtruth : associated segmentation map rotated

    '''
    # Shape of the image on which the segmentation map is placed
    # The shape of the segmentation map array is identical to the image shape
    shape_image = groundtruth.shape  # (400,400)

    # Object representing a segmentation map associated with an image
    seg_map =  SegmentationMapsOnImage(groundtruth, shape=shape_image)
    
    
    seq = iaa.Affine(rotate=degree, # Rotate image by a certain degree
                     mode='symmetric') # Pads with the reflection of the vector mirrored along the edge of the array
                        
    seq._mode_segmentation_maps = 'symmetric'
    
    # Augmentation
    augmented_image, augmented_groundtruth = seq(image=image, segmentation_maps=seg_map)
    # The augmented segmentation map is converted back to an array
    augmented_groundtruth = 255 * augmented_groundtruth.get_arr().astype('uint8')

    return augmented_image, augmented_groundtruth

In [24]:
def data_augmentation_blur(image, groundtruth, blur = 3):
    '''Create new image by bluring the image 
    Arguments : 
        - image : input image of shape (H,W) that we want to augment
        - groundtruth : segmentation map of the corresponding input image
        - blur : value of the degree of bluring
    Return :
        - augmented_image : images blured 
        - augmented_groundtruth : associated segmentation map blured

    '''
    # Shape of the image on which the segmentation map is placed
    # The shape of the segmentation map array is identical to the image shape
    shape_image = groundtruth.shape  # (400,400)

    # Object representing a segmentation map associated with an image
    seg_map =  SegmentationMapsOnImage(groundtruth, shape=shape_image)
    
    
    seq = iaa.GaussianBlur((0.0, blur))  #bluring the images  
   
    # Augmentation
    augmented_image, augmented_groundtruth = seq(image=image, segmentation_maps=seg_map)
    # The augmented segmentation map is converted back to an array
    augmented_groundtruth = 255 * augmented_groundtruth.get_arr().astype('uint8')

    return augmented_image, augmented_groundtruth

In [25]:
def data_augmentation_crop(image, groundtruth, crop = 0.1):
    '''Create new image by bluring the image 
    Arguments : 
        - image : input image of shape (H,W) that we want to augment
        - groundtruth : segmentation map of the corresponding input image
        - crop : value of the degree of croping
    Return :
        - augmented_image : images croped 
        - augmented_groundtruth : associated segmentation map croped

    '''
    # Shape of the image on which the segmentation map is placed
    # The shape of the segmentation map array is identical to the image shape
    shape_image = groundtruth.shape  # (400,400)

    # Object representing a segmentation map associated with an image
    seg_map =  SegmentationMapsOnImage(groundtruth, shape=shape_image)
    
    
    seq = iaa.Crop(percent=crop)  #cropping the images  
   
    # Augmentation
    augmented_image, augmented_groundtruth = seq(image=image, segmentation_maps=seg_map)
    # The augmented segmentation map is converted back to an array
    augmented_groundtruth = 255 * augmented_groundtruth.get_arr().astype('uint8')

    return augmented_image, augmented_groundtruth

In [26]:
def data_augmentation_shear(image, groundtruth, shear = 20):
    '''Create new image by shearing the image 
    Arguments : 
        - image : input image of shape (H,W) that we want to augment
        - groundtruth : segmentation map of the corresponding input image
        - crop : value of the degree of shearing
    Return :
        - augmented_image : images sheared 
        - augmented_groundtruth : associated segmentation map sheared

    '''
    # Shape of the image on which the segmentation map is placed
    # The shape of the segmentation map array is identical to the image shape
    shape_image = groundtruth.shape  # (400,400)

    # Object representing a segmentation map associated with an image
    seg_map =  SegmentationMapsOnImage(groundtruth, shape=shape_image)
    
    shear_ = shear
    seq = iaa.Affine(shear= shear_) #shearing the images  
   
    # Augmentation
    augmented_image, augmented_groundtruth = seq(image=image, segmentation_maps=seg_map)
    # The augmented segmentation map is converted back to an array
    augmented_groundtruth = 255 * augmented_groundtruth.get_arr().astype('uint8')

    return augmented_image, augmented_groundtruth

### Data augmentation total

In [27]:
def data_augmented(images, groundtruths):
    '''Augment the number of input images by rotation and flip
    Arguments :
         - images : list of image
         - groundtruths : list of groundtruth
    '''

    n = len(images)

    for i in range(n):
        augmented_image, augmented_groundtruth = data_augmentation_rotation(images[i], groundtruths[i], degree = 90)
        augmented_image_, augmented_groundtruth_ = data_augmentation_rotation(images[i], groundtruths[i], degree = 45)
        flipped_image_h, flipped_groundtruth_h = data_augmentation_flip(images[i], groundtruths[i], horonzital = True)
        flipped_image_v, flipped_groundtruth_v = data_augmentation_flip(images[i], groundtruths[i], horonzital = False)
        augmented_image_shear, augmented_groundtruth_shear = data_augmentation_shear(images[i], groundtruths[i], shear = 20)
        augmented_image_crop, augmented_groundtruth_crop = data_augmentation_crop(images[i], groundtruths[i], crop = 0.3)

        # Saving the new images and groundtruths
        save_data_augmented(augmented_image, augmented_groundtruth)
        save_data_augmented(flipped_image_h, flipped_groundtruth_h)
        save_data_augmented(flipped_image_v, flipped_groundtruth_v)
        save_data_augmented(augmented_image_, augmented_groundtruth_)
        save_data_augmented(augmented_image_shear, augmented_groundtruth_shear)
        save_data_augmented(augmented_image_crop, augmented_groundtruth_crop)



In [28]:
data_augmented(imgs, gt_imgs)

c:\Users\pauli\miniconda3\Lib\site-packages\imgaug\imgaug.py:106: DeprecationWarning: Got a float array as the segmentation map in SegmentationMapsOnImage. That is deprecated. Please provide instead a (H,W,[C]) array of dtype bool_, int or uint, where C denotes the segmentation map index.
  warn(msg, category=DeprecationWarning, stacklevel=stacklevel)
